In [1]:
import random
import numpy as np

In [56]:
num_points = 100
image_width = 640
image_height = 480
baseline = 0.2
focal_length = 931.205

cam_K = np.array([[focal_length, 0.0, image_width/2],
                  [0.0, focal_length, image_height/2],
                  [0.0, 0.0, 1.0]])

right_cam_P = np.linalg.inv(np.array([[1.0, 0.0, 0.0, baseline],
                                      [0.0, 1.0, 0.0, 0.0],
                                      [0.0, 0.0, 1.0, 0.0],
                                      [0.0, 0.0, 0.0, 1.0]]))
l_kps = []
r_kps = []
features = []

for _ in range(num_points):
    x = random.randint(0, image_width-1)
    y = random.randint(0, image_height-1)
    depth = random.uniform(2.0, 6.0)
    
    # save left keypoint
    l_kp = [[x], [y]]
    l_kps.append(l_kp)
    
    # collate homogenous keypoint
    l_h_kp = [l_kp[0], l_kp[1], [1]]

    # project keypoint to normalised landmark
    l_ft = np.dot(np.linalg.inv(cam_K), l_h_kp)
    l_ft *= depth
    features.append(l_ft)

    # collate homogenous landmark
    l_h_ft = np.append(l_ft, [[1]], axis=0)

    # transform point wrt right camera coordinates
    r_h_ft = np.dot(right_cam_P, l_h_ft)
    r_h_ft /= r_h_ft[2]

    # re-project landmark into right camera
    r_h_kp = np.dot(cam_K, r_h_ft[:3,:])
    r_kp = [r_h_kp[0], r_h_kp[1]]
    r_kps.append(r_kp)


with open("./ba_problem.txt", "w") as ba_file:
    ba_file.write("2 " + str(len(features)) + " " + str(len(l_kps) + len(r_kps)) + "\n")
    
    for id, kp in enumerate(l_kps):
        ba_file.write("0 " + str(id) + " " + str(kp[0][0])+ " " + str(kp[1][0]) + "\n")
    
    for id, kp in enumerate(r_kps):
        ba_file.write("1 " + str(id) + " " + str(kp[0][0])+ " " + str(kp[1][0]) + "\n")
    
    ba_file.write("0.0\n0.0\n0.0\n0.0\n0.0\n0.0\n" + str(focal_length) + "\n0.0\n0.0\n")
    ba_file.write("0.0\n0.0\n0.0\n" + str(baseline) + "\n0.0\n0.0\n" + str(focal_length) + "\n0.0\n0.0\n")
    
    for feature in features:
        ba_file.write(str(feature[0][0]) + "\n" + str(feature[1][0]) + "\n" + str(feature[2][0]) + "\n")

In [ ]:
# +static void write_problem(  const string& file_path,
# +                            const std::vector< double >& out_init3DPoints,
# +                            const std::vector< std::vector< cv::Point2f > >& vv_imgPoints)
# +{
# +  ofstream myfile (file_path);
# +  if (myfile.is_open())
# +  {
# +    myfile << vv_imgPoints.size() << " " << int(out_init3DPoints.size()/3) << " " << vv_imgPoints[0].size()+vv_imgPoints[1].size() << "\n";
# +    for(int cam_id = 0; cam_id<vv_imgPoints.size(); ++cam_id){
# +        for(int feat_id = 0; feat_id<vv_imgPoints[cam_id].size(); ++feat_id){
# +            myfile << cam_id << " " << feat_id << " " << vv_imgPoints[cam_id][feat_id].x << " " << vv_imgPoints[cam_id][feat_id].y << "\n";
# +        }
# +    }
# +    myfile << "0.0\n0.0\n0.0\n0.0\n0.0\n0.0\n931.205\n0.0\n0.0\n";
# +    myfile << "0.0\n0.0\n0.0\n0.035\n0.0\n0.0\n931.205\n0.0\n0.0\n";
# +    for(int pt_id = 0; pt_id<out_init3DPoints.size(); ++pt_id){
# +        myfile <<  out_init3DPoints[pt_id] << "\n";
# +    }
# +    myfile.close();
# +  }
# +  else cout << "Unable to open file";
# +}
# +